# Fast R-CNN

In [1]:
!nvidia-smi

Fri Apr  1 21:36:07 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 418.67       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:43:00.0 Off |                    0 |
| N/A   33C    P0    41W / 300W |      0MiB / 32480MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

## Preparing

In [2]:
# install PaddleX
# ! pip install paddlex

In [3]:
import paddle
import paddlex as pdx
import numpy as np

paddle.__version__, paddle.device.get_device(), pdx.__version__

[04-01 21:36:09 MainThread @utils.py:79] WRN paddlepaddle version: 2.2.2. The dynamic graph version of PARL is under development, not fully tested and supported


/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/parl/remote/communication.py:38: DeprecationWarning: 'pyarrow.default_serialization_context' is deprecated as of 2.0.0 and will be removed in a future version. Use pickle or the pyarrow IPC functionality instead.
  context = pyarrow.default_serialization_context()
/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/matplotlib/__init__.py:107: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import MutableMapping
/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/matplotlib/rcsetup.py:20: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import Iterable, Mapping
/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/matplotlib/colors.py:53: Depreca

('2.2.2', 'gpu:0', '2.1.0')

In [4]:
import paddlex as pdx
# from paddlex import transforms as T
from paddlex import transforms as T

# download and unzip an insect recognition dataset
dataset = 'https://bj.bcebos.com/paddlex/datasets/insect_det.tar.gz'
pdx.utils.download_and_decompress(dataset, path='./')

train_transforms = T.Compose([
    T.RandomResizeByShort(
        short_sizes=[640, 672, 704, 736, 768, 800],
        max_size=1333,
        interp='CUBIC'), T.RandomHorizontalFlip(), T.Normalize(
            mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

eval_transforms = T.Compose([
    T.ResizeByShort(
        short_size=800, max_size=1333, interp='CUBIC'), T.Normalize(
            mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

2022-04-01 21:36:12 [INFO]	Downloading insect_det.tar.gz from https://bj.bcebos.com/paddlex/datasets/insect_det.tar.gz


100%|██████████| 16899/16899 [00:00<00:00, 54458.13KB/s]


2022-04-01 21:36:12 [INFO]	Decompressing ./insect_det.tar.gz...


## Dataset

In [5]:
train_dataset = pdx.datasets.VOCDetection(
    data_dir='insect_det',
    file_list='insect_det/train_list.txt',
    label_list='insect_det/labels.txt',
    transforms=train_transforms,
    shuffle=True)

eval_dataset = pdx.datasets.VOCDetection(
    data_dir='insect_det',
    file_list='insect_det/val_list.txt',
    label_list='insect_det/labels.txt',
    transforms=eval_transforms,
    shuffle=False)

2022-04-01 21:36:13 [INFO]	Starting to read file list from dataset...
2022-04-01 21:36:13 [INFO]	169 samples in file insect_det/train_list.txt, including 169 positive samples and 0 negative samples.
creating index...
index created!
2022-04-01 21:36:13 [INFO]	Starting to read file list from dataset...
2022-04-01 21:36:13 [INFO]	24 samples in file insect_det/val_list.txt, including 24 positive samples and 0 negative samples.
creating index...
index created!


In [6]:
num_classes = len(train_dataset.labels)
print("num of labels is:", num_classes)
model = pdx.det.FasterRCNN(num_classes=num_classes, backbone='ResNet101_vd', with_fpn=True)

num of labels is: 6


W0401 21:36:13.906572   836 device_context.cc:447] Please NOTE: device: 0, GPU Compute Capability: 7.0, Driver API Version: 10.1, Runtime API Version: 10.1
W0401 21:36:13.910257   836 device_context.cc:465] device: 0, cuDNN Version: 7.6.


## Training

In [7]:
model.train(
    num_epochs=13,
    train_dataset=train_dataset,
    train_batch_size=2,
    eval_dataset=eval_dataset,
    learning_rate=0.0025,
    lr_decay_epochs=[8, 11],
    warmup_steps=500,
    warmup_start_lr=0.00025,
    # save_interval_epochs=1,
    save_dir='output/faster_rcnn_r50_fpn',
    use_vdl=True)

2022-04-01 21:36:16 [INFO]	Downloading ResNet101_vd_pretrained.pdparams from https://paddledet.bj.bcebos.com/models/pretrained/ResNet101_vd_pretrained.pdparams


100%|██████████| 166559/166559 [00:02<00:00, 70994.87KB/s]


2022-04-01 21:36:18 [INFO]	Loading pretrained model from output/faster_rcnn_r50_fpn/pretrain/ResNet101_vd_pretrained.pdparams
2022-04-01 21:36:19 [WARNING]	neck.fpn_inner_res2_sum_lateral.weight is not in pretrained model
2022-04-01 21:36:19 [WARNING]	neck.fpn_inner_res2_sum_lateral.bias is not in pretrained model
2022-04-01 21:36:19 [WARNING]	neck.fpn_res2_sum.weight is not in pretrained model
2022-04-01 21:36:19 [WARNING]	neck.fpn_res2_sum.bias is not in pretrained model
2022-04-01 21:36:19 [WARNING]	neck.fpn_inner_res3_sum_lateral.weight is not in pretrained model
2022-04-01 21:36:19 [WARNING]	neck.fpn_inner_res3_sum_lateral.bias is not in pretrained model
2022-04-01 21:36:19 [WARNING]	neck.fpn_res3_sum.weight is not in pretrained model
2022-04-01 21:36:19 [WARNING]	neck.fpn_res3_sum.bias is not in pretrained model
2022-04-01 21:36:19 [WARNING]	neck.fpn_inner_res4_sum_lateral.weight is not in pretrained model
2022-04-01 21:36:19 [WARNING]	neck.fpn_inner_res4_sum_lateral.bias is not 


### Metric curve from VisualDL

![](https://ai-studio-static-online.cdn.bcebos.com/bfca8aa4f75c46c293eba6be9bba5614c554eaee21d54047ab95d413ea429ff3)
![](https://ai-studio-static-online.cdn.bcebos.com/52f2fbe2e20f46b8b35d046763a05c5966f45a3dd491484ba8666dda73137917)
![](https://ai-studio-static-online.cdn.bcebos.com/5bb2713606a043b0ad81eb3a4a6831605d648c963972479f9043df019901583c)



## Prediction

In [8]:
import paddlex as pdx
model = pdx.load_model('output/faster_rcnn_r50_fpn/best_model')
image_name = 'insect_det/JPEGImages/0216.jpg'
result = model.predict(image_name)

pdx.det.visualize(image_name, result, threshold=0.5, save_dir='./output/faster_rcnn_r50_fpn')

2022-04-01 21:41:07 [INFO]	Model[FasterRCNN] loaded.
2022-04-01 21:41:07 [INFO]	The visualized result is saved at ./output/faster_rcnn_r50_fpn/visualize_0216.jpg


### 0216.jpg
![](https://ai-studio-static-online.cdn.bcebos.com/e7f192edc43741b99dbd07c171fa2447307d536025984460ba5702860532f237)


In [9]:
image_name = 'insect_det/JPEGImages/0217.jpg'
result = model.predict(image_name)

pdx.det.visualize(image_name, result, threshold=0.5, save_dir='./output/faster_rcnn_r50_fpn')

2022-04-01 21:41:07 [INFO]	The visualized result is saved at ./output/faster_rcnn_r50_fpn/visualize_0217.jpg


### 0217.jpg
![](https://ai-studio-static-online.cdn.bcebos.com/94801fd9c84b40138de769b41734f0b58aed5a6419264f9fb6bc4572d857e089)


In [10]:
image_name = 'insect_det/JPEGImages/0214.jpg'
result = model.predict(image_name)

pdx.det.visualize(image_name, result, threshold=0.5, save_dir='./output/faster_rcnn_r50_fpn')

2022-04-01 21:41:07 [INFO]	The visualized result is saved at ./output/faster_rcnn_r50_fpn/visualize_0214.jpg


### 0214.jpg
![](https://ai-studio-static-online.cdn.bcebos.com/055428a0eea34d76a58dc0efefaf12e950a271c6a96b4820b9c19e136a864488)
